In [4]:
import torch
import os
os.environ['NEURITE_BACKEND'] = 'pytorch'
os.environ['VXM_BACKEND'] = 'pytorch'
import voxelmorph as vxm
import matplotlib.pyplot as plt
import glob
import re
import csv
from tifffile import imwrite
import numpy as np
from PIL import Image
import fnmatch
from collections import defaultdict
import shutil

from my_utils_v2 import Utils_v2

In [5]:
tissue_path='/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered'
tissue_name='GCA033TIB_TISSUE01'
mask_path=f'/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/{tissue_name}_RetentionMask.tif'
output_path='/fs5/p_masi/rudravg/MxIF_Vxm_Registered'

In [6]:
all_tissues=[]
for file in os.listdir(tissue_path):
    if fnmatch.fnmatch(file, f'{tissue_name}_*'):
        all_tissues.append(os.path.join(tissue_path, file))

In [7]:
all_tissues

['/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_CD3d_CY3_150ms_ROUND_10.tif',
 '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_pEGFR_CY2_175ms_ROUND_08.tif',
 '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_Background_CY5_500ms_ROUND_13.tif',
 '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_DAPI_DAPI_12ms_ROUND_17.tif',
 '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_Background_CY5_100ms_ROUND_15.tif',
 '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_DAPI_DAPI_12ms_ROUND_10.tif',
 '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033TIB_TISSUE01_Background_CY2_500ms_ROUND_05.tif',
 '/fs5/p_masi/baos1/rudravg/MXIF/MXIF/Helmsley/MxIF/Set02/GCA033TIB/Registered/GCA033

In [8]:
filtered_tissues = list(filter(lambda x: 'RetentionMask' not in x, all_tissues))

# Group the strings by round number
tissues_by_round = defaultdict(list)
for tissue in filtered_tissues:
    round_number = int(re.search('ROUND_(\\d+)', tissue).group(1))
    tissues_by_round[round_number].append(tissue)

# Sort the dictionary items by key (round number)
sorted_items = sorted(tissues_by_round.items())

# Sort each group so that the string containing "DAPI_DAPI" comes first, and convert the sorted dictionary items to a list
grouped_tissues = [sorted(group, key=lambda x: f'{tissue_name}_DAPI' not in x) for _, group in sorted_items]

In [9]:
new_dir = os.path.join(output_path, tissue_name,"Unregistered")
os.makedirs(new_dir, exist_ok=True)

In [10]:
for group in grouped_tissues:
    for file_path in group:
        file,height,width=Utils_v2.load_tissues_for_overlap(file_path,mask_path)
        file=file*255.
        file = file[:height, :width]
        file_name=os.path.basename(file_path)[:-4]
        Image.fromarray(np.uint8(file)).save(os.path.join(new_dir,f"{file_name}.tif"))